## Download White Wine Data From Vivino.com

Before running this download the file chromedriver, you will need to enter the path to this file below:

driver = webdriver.Chrome(executable_path='path', chrome_options=options)

if you are using Windows input the path to chromedriver under the if statement:

if os.name == 'nt'

otherwise enter it under the else statement

Chrome 59 or greater is required for the headless option to work.  Script should still run, just not headless.

Libraries:
* conda install -c conda-forge selenium
* https://sites.google.com/a/chromium.org/chromedriver/downloads

Note:  Red wine script included below, but is not completed.  I'm tired and want to go to bed.  Future Chris can deal with it.

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import os
import csv
import pandas as pd

options = webdriver.ChromeOptions(); options.add_argument('headless')
if os.name == 'nt':
    driver = webdriver.Chrome(executable_path='D:\orson\Downloads\chromedriver_win32\chromedriver', chrome_options=options)    
else:
    driver = webdriver.Chrome(executable_path='/home/chris/Downloads/chromedriver', chrome_options=options)

country_list = []; rating_list = []; popular_list = []; price_list = []
wine_list    = []; winery_list = []
site_list    = ['https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NFDLTaywNTRWS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAb34PXA==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NFbLTaywNTRTS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcE0PYg==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NFPLTaywNTJQS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcCUPYA==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MlDLTaywNTJVS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcAMPYA==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MlXLTaywNTVQS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcIYPYw==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NVDLTawAUgZqyZW2pcVqBUCJYtvkRLWyZNuSotJUtfKS6FhbIwB7dQ-R']

driver.get(site_list[0])
# move = webdriver.ActionChains(driver)
# move = ActionChains(driver)
# slider = driver.find_element_by_xpath('//span[@class="ui-slider-handle ui-corner-all ui-state-default"]')

# elements = [driver.find_element_by_xpath('//span[@class="ui-slider-handle ui-corner-all ui-state-default"]'),
#             driver.find_element_by_xpath('//div[@class="query-summary__primary"]'),
#             driver.find_element_by_xpath('//span[@class="wine-explorer__filters__item__header__title"]')]
# for item in elements:
#     time.sleep(5)
#     print("Test")
#     move.move_to_element(item).perform()
# move.click_and_hold(slider).move_by_offset(20, 0).release()

# time.sleep(5)
# move.click_and_hold(slider

# time.sleep(5)
# move.click_and_hold(slider).move_by_offset(20, 0).release()

for site in range(len(site_list)):
    print("Starting Scrape of {0} of {1} Websites".format(site+1, len(site_list)))
    driver.get(site_list[site])

    lastHeight = driver.execute_script("return document.body.scrollHeight")
    total_time = 0; total_page_loads = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight and total_time > 10:
            print('  No new data on this site'); break
        time.sleep(0.25); total_time += 0.25
        if lastHeight < newHeight:
            if total_page_loads % 5 == 0:
                print("  Pages Loaded:   ", total_page_loads)
            total_page_loads += 1; total_time = 0; lastHeight = newHeight

    time.sleep(5.0)
    price_elements      = driver.find_elements_by_xpath('//div[@class="wine-card__statistics"]/div[2]/div[2]/div[1]/span[1]')
    country_elements    = driver.find_elements_by_xpath('//div[@class="location"]/a[2]')
    popularity_elements = driver.find_elements_by_xpath('//div[@class="statistics-item__content__description"]/div[2]/span[1]/span[1]')
    rating_elements     = driver.find_elements_by_xpath('//div[@class="wine-card__statistics"]/div[1]/div[2]/div[1]')
    wine_elements       = driver.find_elements_by_xpath('//a[@class="wine-card__header"]/span[2]')
    winery_elements     = driver.find_elements_by_xpath('//a[@class="wine-card__header"]/span[1]')

    prices       = [x.text for x in price_elements]
    countries    = [x.text for x in country_elements]
    popularities = [x.text for x in popularity_elements]
    ratings      = [x.text for x in rating_elements]
    wineries     = [x.text for x in winery_elements]
    wines        = [x.text for x in wine_elements]
    number       = driver.find_element_by_xpath('//div[@class="query-summary__primary"]/span[3]').text

    print("Sanity check should find ", number, " wines in each list")
    print(len(countries)); 
    while len(countries) < int(number):
        countries.append('')
    country_list = country_list + countries
    print(len(ratings));    
    while len(ratings) < int(number):
        ratings.append('')
    rating_list  = rating_list  + ratings
    print(len(prices));     
    while len(prices) < int(number):
        prices.append('')
    price_list   = price_list   + prices
    print(len(popularities)); 
    while len(popularities) < int(number):
        popularities.append('')
    popular_list = popular_list + popularities
    print(len(wines))
    while len(wines) < int(number):
        wines.append('')
    wine_list = wine_list + wines
    print(len(wineries))
    while len(wineries) < int(number):
        wineries.append('')
    winery_list = winery_list + wineries


outputter = pd.DataFrame({'wine':wine_list, 'winery':winery_list, 'country':country_list, 'rating':rating_list, \
                          'price':price_list, 'popularity':popular_list})
outputter.to_csv('vivino_white_wine.csv')


print("\n\nThat's all Folks!")


Starting Scrape of 1 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  No new data on this site
Sanity check should find  465  wines in each list
464
465
465
465
465
465
Starting Scrape of 2 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  Pages Loaded:    20
  No new data on this site
Sanity check should find  529  wines in each list
528
529
529
529
529
529
Starting Scrape of 3 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  Pages Loaded:    20
  Pages Loaded:    25
  No new data on this site
Sanity check should find  617  wines in each list
617
617
617
617
617
617
Starting Scrape of 4 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  No new data on this site
Sanity check should find  427  wines in each list
427
427
427
427
427
427
Starting Scrape of 5 of 6 Websites
  Pages Loa

In [83]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import os
import csv
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('headless')
if os.name == 'nt':
    driver = webdriver.Chrome(executable_path='D:\orson\Downloads\chromedriver_win32\chromedriver')
else:
    driver = webdriver.Chrome(executable_path='/home/chris/Downloads/chromedriver', chrome_options=options)
driver.get('https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgIAcDcPWw==')

lastHeight = driver.execute_script("return document.body.scrollHeight")
total_time = 0
total_page_loads = 0
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    newHeight = driver.execute_script("return document.body.scrollHeight")
    print(total_time, "    ", newHeight, "    ", lastHeight)
    if total_page_loads % 10 == 0:
        print("Pages Loaded     : ", total_page_loads)
    if newHeight == lastHeight and total_time > 30:
        print('No New Data')
        break
    time.sleep(0.25)
    total_time += 0.25
    if lastHeight < newHeight:
        total_page_loads += 1
        total_time = 0
        lastHeight = newHeight

time.sleep(10.0)
price_element = driver.find_elements_by_xpath('//div[@class="wine-card__statistics"]/div[2]/div[2]/div[1]/span[1]')
prices = [x.text for x in price_element]
countries_element = driver.find_elements_by_xpath('//div[@class="location"]/a[2]')
countries = [x.text for x in countries_element]
popularity_element = driver.find_elements_by_xpath('//div[@class="statistics-item__content__description"]/div[2]/span[1]/span[1]')
popularity = [x.text for x in popularity_element]
ratings_element = driver.find_elements_by_xpath('//div[@class="wine-card__statistics"]/div[1]/div[2]/div[1]')
ratings = [x.text for x in ratings_element]
number = driver.find_element_by_xpath('//div[@class="query-summary__primary"]/span[3]').text


print("Sanity check should find ", number, " wines in each list")
print(len(countries))
print(len(ratings))
print(len(prices))
print(len(popularity))

outputter = pd.DataFrame({'country':countries, 'rating':ratings, \
                          'price':prices, 'popularity':popularity})
outputter.to_csv('vivino_red_wine.csv')


print("\n\nThat's all Folks!")


0      10465      10465
Pages Loaded     :  0
0.25      19514      10465
Pages Loaded     :  0
0      29439      19514
0      30439      29439
0      39364      30439
0      49165      39364
0      69015      49165
0      70139      69015
0      88865      70139
0      98790      88865
0      99914      98790
0      118640      99914
Pages Loaded     :  10
0      128565      118640
0      139490      128565
0      149415      139490
0      159340      149415
0      159464      159340
0      179190      159464
0      189115      179190
0      199040      189115
0      208965      199040
0      218890      208965
Pages Loaded     :  20
0      228815      218890
0      238740      228815
0      247665      238740
0      257590      247665
0      268515      257590
0      278440      268515
0      288365      278440
0      298290      288365
0      308215      298290
0      318140      308215
Pages Loaded     :  30
0      328065      318140
0      337990      328065
0      347915      3379

KeyboardInterrupt: 